# Simulating Network Deliberation 

In [ ]:
# Configuration

N = 100
M = 5
runs = 100
stages = 25
steps = 1

bit_count = 7
p_error = 0.4

small_world_k = 4
small_world_a = 0.1
erdos_renyi_p = 0.01
barabasi_albert_m = 2

In [ ]:
# Configure plotting in Jupyter
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams.update({
    'figure.figsize': (20, 20),
    'axes.spines.right': False,
    'axes.spines.left': False,
    'axes.spines.top': False,
    'axes.spines.bottom': False})

# Imports
import random

import networkx as nx
import numpy as np
from numpy import random as nprand
from tqdm.notebook import tqdm

import netdelib.soclearn as slearn
import netdelib.soclearn.evaluate as sleval
import netdelib.soclearn.models.generated as slgen
import netdelib.soclearn.strategy as slstrat
import netdelib.topologies.topologies as topo

## Generates topologies

In [ ]:
class NetworkFactory(object):
    """Base class for network factories.
    
    Constructor paramteters
    N: Number of participants
    M: Number of participants per group
    stage_graphs: Whether to regenerate network at each stage
    """

    def __init__(self, N, M, stage_graphs=True):
        self.N = N
        self.M = M
        self.stage_graphs = stage_graphs
    
    def create(self, stage):
        """Create a networkx Graph with a particular topology.
        
        Parameters
        stage: The deliberation stage
        
        Returns
        A networkx Graph.
        """
        return nx.Graph()

class LongPathFactory(NetworkFactory):
    '''Factory class for long-path networks.'''
    def create(self, stage):
        groups = topo.get_long_path_stage_groups(self.N, self.M, stage)
        G = nx.Graph()
        for group in groups:
            g = nx.complete_graph(group) #clique network
            G = nx.union(G, g)
        return G
    
class RandomGroupFactory(NetworkFactory):
    """Factory class for random group networks."""
    def create(self, stage):
        groups = topo.get_random_stage_groups(self.N, self.M, stage)
        G = nx.Graph()
        for group in groups:
            g = nx.complete_graph(group) #clique network
            G = nx.union(G, g)
        return G

class CompleteFactory(NetworkFactory):
    """Factory class for deliberation network with random group assignments."""

    def __init__(self, N, M):
        super(CompleteFactory, self).__init__(N, M, False)
        
    def create(self, stage):
        return nx.complete_graph(self.N)

class PreferentialFactory(NetworkFactory):
    """Factory class for preferential attachment networks.
    
    Constructor Params
    N: Number of participants
    M: Number of participants per group (unused)
    m: Number of edges per node"""
    
    def __init__(self, N, M, m):
        super(PreferentialFactory, self).__init__(N, M, False)
        self.m = m
        
    def create(self, stage):
        return nx.barabasi_albert_graph(self.N, self.m)
    
class SmallWorldFactory(NetworkFactory):
    """Factory class for small-world networks.
    
    Constructor Params
    N: Number of participants
    M: Number of participants per group (unused)
    k: Number of edges per node
    a: probability of rewiring
    """
    
    def __init__(self, N, M, k, a):
        super(SmallWorldFactory, self).__init__(N, M, False)
        self.k = k
        self.a = a
        
    def create(self, stage):
        return nx.watts_strogatz_graph(self.N, self.k, self.a)
    
class RandomFactory(NetworkFactory):
    """Factory class for Erdos-Renyi random networks.
    
    Constructor Params
    N: Number of participants
    M: Number of participants per group (unused)
    p: Probability of edge existing between two nodes
    """
    
    def __init__(self, N, M, p):
        super(RandomFactory, self).__init__(N, M, False)
        self.p
        
    def create(self, stage):
        return nx.erdos_renyi_graph(N, self.p)


## run_trial

In [ ]:
def run_trial(
    factory,
    p_error,
    learning_strategy,
    true_value,
    runs,
    N,
    M,
    stages,
    steps
):
    """
    Plots the fraction of correct nodes in all stages of a simulation
    
    #Parameters:
    factory: network factory
    learning strategy: learning strategy to simulate in topology
    intial beliefs are generated inside the topology: prarms (G, true_value, p_error)
    true_value: the ground truth 
    runs: the number of runs 
    stage_graphs: checks if the graph needs to be change in subsequent stages
    stages: for now is fixed, eventually will change. (fixed for now) 
    
    return
    a dictrionary of lists. the keys are the number of the run, and the val is the fract of nodes with correct beliefs
    """

    #keep values of each run: key: # run val: plot_belief_bits_correct
    runs_list_fract_corrrect = dict(
        (run, list())
        for run in range(runs)) #for each bit creates an empty list. 
    
    for run in range(runs):
        beliefs_stages = []
    
        for stage in range(stages):

            if stage == 0: 
                G = factory.create(stage)
                ini_beliefs = slgen.initial_beliefs_noisy(G, true_value, p_error=p_error)
                beliefs_stages = [ini_beliefs]
            else:
                if factory.stage_graphs:
                    G = factory.create(stage)
                ini_beliefs = beliefs_stages[-1]
                
            beliefs_list = slearn.learn(G, ini_beliefs, learning_strategy, true_value, steps)
            beliefs_stages += beliefs_list[1:]
            
        runs_list_fract_corrrect[run] += sleval.beliefs_correct(beliefs_stages, true_value)
    return runs_list_fract_corrrect

In [ ]:
def sta_des_all_runs(factory, p_error, learning_strategy, true_value, runs, N, M, stages, steps):
    '''Takes the values of all runs and finds the mean and std dev
    #Parameters 
    factory: network factory 
    learning_strategy: learning strategy to simulate in topology
    true_value: the ground truth 
    runs: the number of runs 
    
    #Returns a plot of mean value of all the runs with a bar error
    '''
    runs_list_fract_corrrect = run_trial(
        factory,
        p_error,
        learning_strategy, 
        true_value,
        runs,
        N, M, stages, steps)

    #runs && num_stpes are use to iterate through each step 
    runs = list(runs_list_fract_corrrect.keys())
    #total num of steps each run
    num_steps = len(runs_list_fract_corrrect[runs[0]])

    #keep values of all the values in each index: key: index; val: fraction of nodes
    fract_corrrect_step = dict(
        (step, list())
        for step in range(num_steps)) #for each bit creates an empty list. 

    for run in runs_list_fract_corrrect:
        for step in range(num_steps):
            #gets the fraction of nodes at step [step]
            #appends the raction of nodes at step [step] to fract_corrrect_step[step] dictrionary
            fract_corrrect_step[step].append(runs_list_fract_corrrect[run][step])

    y = []
    y_error = []
    for step in fract_corrrect_step:
        mean_each_step = np.mean(fract_corrrect_step[step])
        sd_each_step = np.std(fract_corrrect_step[step])
        
        y.append(mean_each_step)
        y_error.append((1.96*sd_each_step/len(fract_corrrect_step[step])**.5))
    
    x = range(len(y))
    y_high = np.array(y) + np.array(y_error)
    y_low = np.array(y) - np.array(y_error)
    plt.fill_between(x, y_low, y_high, color="#aaaaaa")
    plt.plot(x, y, 'b')

    #add spines to plot
    ax = plt.gca()
    for spine in ax.spines.values():
        spine.set_visible(True)
    plt.xlim([0, len(x)])
    plt.ylim([0, 1.1])

In [ ]:
networks = {
    'Complete': CompleteFactory(N, M),
    'Pref. Attach.': PreferentialFactory(N, M, barabasi_albert_m),
    'Small World': SmallWorldFactory(N, M, small_world_k, small_world_a),
    'Long Path': LongPathFactory(N, M),
    'Random Group': RandomGroupFactory(N, M)   
}

In [ ]:
learning_strategies = {
    'Best Neighbor': slstrat.best_neighbor,
    #'Random bit': random_neighbor_bit,
    #'Random list': random_neighbor_list,
    'Conform': slstrat.conform,
    'Local Majority': slstrat.local_majority,
}

# Simulations 

In [ ]:
def run(bit_count, p_error, runs, N, M, stages, steps):
    for j, (title, factory) in tqdm(enumerate(networks.items())):
        for i, (title2, learning_strategy) in enumerate(learning_strategies.items()):
                plt.subplot(
                    len(learning_strategies),
                    len(networks),
                    1 + i * len(networks) + j)
                plt.title(f'{title} - {title2}')
                if j == 0:
                    plt.ylabel('Frac. Correct')
                if i == 2:
                    plt.xlabel('Time Step')
                plt.grid(True)
                sta_des_all_runs(
                    factory,
                    p_error,
                    learning_strategy,
                    tuple([1 for x in range(bit_count)]),
                    runs,
                    N, M, stages, steps)
    plt.tight_layout()

In [ ]:
plt.figure(figsize=(15, 9))
run(bit_count, p_error, runs, N, M, stages, steps)
plt.tight_layout()